In [1]:
import time
import os
import re
import random
# 线程池
from concurrent.futures import ThreadPoolExecutor

import requests
from bs4 import BeautifulSoup

# 视频网站: https://www.dianying111.com/

## 功能介绍:
    爬取上面的视频网站的视频    
    通过视频的播放链接,下载视频,并自动获取视频名称

In [ ]:
class Move_ts_crawler(object):
    my_headers = [
        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.153 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) Gecko/20100101 Firefox/30.0",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_2) AppleWebKit/537.75.14 (KHTML, like Gecko) Version/7.0.3 Safari/537.75.14",
        "Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; Win64; x64; Trident/6.0)",
        'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
        'Opera/9.25 (Windows NT 5.1; U; en)',
        'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
        'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
        'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
        'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9',
        "Mozilla/5.0 (X11; Linux i686) AppleWebKit/535.7 (KHTML, like Gecko) Ubuntu/11.04 Chromium/16.0.912.77 Chrome/16.0.912.77 Safari/535.7",
        "Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0 ",
        'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3314.0 Safari/537.36 SE 2.X MetaSr 1.0']

    # 创建线程池
    pool = ThreadPoolExecutor(50)

    def __init__(self, url, file_name):
        self.url = url
        self.ts_url = None
        self.file_name = file_name
        self.ts_file_name = file_name + 'BloodShot/'
        self.video_length = 0
        self.video_name = None
        self.header = {'User-Agent': Move_ts_crawler.my_headers[random.randrange(0, len(Move_ts_crawler.my_headers))]}
        self.__check_file()

    def __check_file(self):
        """
        检查文件目录是否创建
        """
        if not os.path.exists(self.file_name):
            os.makedirs(self.file_name)

        if os.path.exists(self.ts_file_name):
            cmd = r'rd /s/q ' + self.ts_file_name.replace('/', '\\')
            os.system(cmd)

        os.makedirs(self.ts_file_name)
        if os.path.exists(self.ts_file_name + 'a.txt'):
            os.remove(self.ts_file_name + 'a.txt')

    def resolve_address(self, url, header):
        """
            m3u8文件url解密
        :param url: 视频链接
        :param header: 请求头
        :return: content: m3u8文件内容
        """
        response = requests.get(url, header)
        soup = BeautifulSoup(response.content, 'lxml')
        url_div = soup.find('div', attrs={'id': 'bofang_box'}).script.text
        pattern = re.compile(r'"url":"(.*?)"')
        m3u8_url = re.findall(pattern, url_div)[0]
        pattern = re.compile(r'正在播放《(.*)》')
        video_name = pattern.findall(response.content.decode('utf8'))[0]
        self.video_name = re.sub(r'\W', '', video_name)
        # 地址解析字典
        dic = {'Mw': '0', 'Mx': '1', 'My': '2', 'Mz': '3', 'M0': '4', 'M1': '5', 'M2': '6', 'M3': '7', 'M4': '8',
               'M5': '9',
               'Yx': 'a', 'Yy': 'b', 'Yz': 'c', 'Y0': 'd', 'Y1': 'e', 'Y2': 'f', 'Y3': 'g', 'Y4': 'h', 'Y5': 'i',
               'ZB': 'j',
               'ZC': 'k', 'ZD': 'l', 'ZE': 'm', 'ZF': 'n', 'ZG': 'o', 'cw': 'p', 'cx': 'q', 'cy': 'r', 'cz': 's',
               'c0': 't',
               'c1': 'u', 'c2': 'v', 'c3': 'w', 'c4': 'x', 'c5': 'y', 'dB': 'z',
               'Qx': 'A', 'Qy': 'B', 'Qz': 'C', 'Q0': 'D', 'Q1': 'E', 'Q2': 'F', 'Q3': 'G', 'Q4': 'H', 'Q5': 'I',
               'RB': 'J',
               'RC': 'K', 'RD': 'L', 'RE': 'M', 'RF': 'N', 'RG': 'O', 'UW': 'P', 'Ux': 'Q', 'Uy': 'R', 'Uz': 'S',
               'U0': 'T',
               'U1': 'U', 'U2': 'V', 'U3': 'W', 'U4': 'X', 'U5': 'Y', 'VB': 'Z',
               'NB': ':', 'JG': '/', 'JF': '.', 'VG': '_', 'JE':'-', '': ''}
        # 将m3u8加密地址拆分
        url_list = re.split('JT', m3u8_url)
        # 存放解密后的地址元素
        new_url = ''
        for i in url_list:
            new_url += dic[i]
        response = requests.get(new_url, header)
        content = response.content.decode('utf8')
        if 'ts' in content:
            ts_url = new_url.replace('/index.m3u8', '')
            return content, new_url
        else:
            m3u8_list = re.split('\n', content)
            if '' in m3u8_list:
                m3u8_list.remove('')
            m3u8_url = m3u8_list[-1]

            if m3u8_url[0] != '/':
                new_url = new_url.replace('index.m3u8', m3u8_url)
                ts_url = new_url.replace('/index.m3u8','')
            else:
                pattern = r'http.*com'
                url_left = re.findall(pattern, new_url)[0]
                new_url = url_left + m3u8_url
                ts_url = url_left
            response = requests.get(new_url)
            content = response.content.decode('utf8')
            return content, ts_url

    def down(self, url, name):
        """
        下载ts文件
        :param url: ts文件url
        :param name: ts文件保存本地名称
        """
        self.header = {'User-Agent': Move_ts_crawler.my_headers[random.randrange(0, len(Move_ts_crawler.my_headers))]}
        response = requests.get(url, self.header)
        if response.status_code != 200:
            print("!!!!!!错误" + str(response.status_code) + url)
            quit()
        with open(self.ts_file_name + name, 'wb') as f:
            f.write(response.content)
        f.close()
        response.close()

    def merge_files(self):
        """
        合并所有ts文件为mp4文件
        """
        print('开始合并文件...')
        cmd = r'ffmpeg -y -f concat -safe 0 -i ' + self.ts_file_name + 'a.txt -c copy ' + self.file_name + 'video.mp4'
        if os.system(cmd):
            print('合并失败')
            print(cmd)
        else:
            print('合并成功,删除其他文件...')
            self.__del_files()

    def __del_files(self):
        """
        删除暂存ts文件的文件夹,同时修改mp4文件名
        """

        cmd = r'rd /s/q ' + self.ts_file_name.replace('/', '\\')
        os.system(cmd)

        src = self.file_name + 'video.mp4'
        dst = self.file_name + self.video_name + '.mp4'
        if os.path.exists(dst):
            os.remove(dst)
        os.rename(src, dst)
        print('删除完成,视频制作完成')
        print('视频地址: ' + dst.replace('/', '\\'))

    def play_video(self):
        """
            是否播放视频
        """
        x = input('你想现在播放吗?y/N')
        while True:
            if x in ['y', 'Y', 'n', 'N']:
                if x in ['y', 'Y']:
                    cmd = 'start '+self.file_name + self.video_name + '.mp4'
                    os.system(cmd)
                    break
                else:
                    break
            else:
                print('输入有误重新输入')
                x = input('你想现在播放吗?y/N')

    def start(self):
        print('开始下载>>>>>')
        content, ts_url = self.resolve_address(self.url, self.header)
        self.ts_url = ts_url
        patten = re.compile(r'\n(.*?)\d+\.ts')
        prefix = patten.search(content)
        if prefix:
            prefix = prefix.group(1)
        self.prefix = prefix
        patten = re.compile(r'\d+\.ts')
        name_list = patten.findall(content)
        self.video_length = len(name_list)  # 获取文件总长度
        print('共%s项'%self.video_length)
        for name in name_list:
            ts_url = self.ts_url + '/' + self.prefix + name
            with open(self.ts_file_name + 'a.txt', 'a+') as f:
                f.write('file ' + name + '\n')
            f.close()
            # 申请线程
            Move_ts_crawler.pool.submit(self.down, ts_url, name)
        print('文件正在下载,请稍后....')

        # 计算下载进度
        schedule_count = 0             #  进度输出控制
        while True:
            # 当前文件个数
            if schedule_count >= 10:
                break
            current_length = len(os.listdir(self.ts_file_name))  # 获取文件当前长度
            schedule = int(current_length / self.video_length * 100)  #　计算百分比
            if schedule >= schedule_count * 10:
                print('文件已下载{0}%'.format(str(schedule)))
                schedule_count += 1

        # 阻塞主线程
        Move_ts_crawler.pool.shutdown(wait=True)
        print('文件下载完成,开始处理....')
        self.merge_files()
        self.play_video()

In [ ]:
if __name__ == '__main__':
    # 视频链接
    url = "https://www.dianying111.com/movieplay/54077-7-1.html"
    # 文件下载地址
    file_name = r'G:/Video/'
    ts = Move_ts_crawler(url, file_name)
    start_time = time.time()
    ts.start()
    end_time = int(round(time.time() - start_time, 0))
    print('下载时长: %s分%s秒' % (str(end_time // 60), str(end_time % 60)))